In [2]:
import cv2
from librerias import filtros, binarizacion
import numpy as np

In [3]:
#comencemos por cargar la imagen en cuestion
imagen = cv2.imread("./src/Jit2.JPG")

In [4]:
#sacamos la escala de grises de la imagen
imagenBN = cv2.cvtColor(imagen,cv2.COLOR_BGR2GRAY)

In [5]:
#apliquemos suavizado para posteriormente umbralar
kernelGauss = filtros.kernelGauss(5, 1)
imagenBNExpandida = filtros.expandirImagen(imagenBN, kernelGauss)
imagenBNSuavizada = filtros.filtrarImagen(imagenBNExpandida, imagenBN, kernelGauss)

[[0.00291502 0.01306423 0.02153928 0.01306423 0.00291502]
 [0.01306423 0.05854983 0.09653235 0.05854983 0.01306423]
 [0.02153928 0.09653235 0.15915494 0.09653235 0.02153928]
 [0.01306423 0.05854983 0.09653235 0.05854983 0.01306423]
 [0.00291502 0.01306423 0.02153928 0.01306423 0.00291502]]
2


In [6]:
#ahora umbralamos con OTSU
histograma = binarizacion.obtenerHistograma(imagenBNSuavizada)
umbral = binarizacion.OTSU(histograma)
imagenUmbralada = binarizacion.umbralar(imagenBNSuavizada, imagen.shape[0], imagen.shape[1], umbral)

varianza de clase: 2587.50299877295
varianza fondo: 0
varianza frente: 2587.50299877295
varianza de clase: 2587.50299877295
varianza fondo: 0
varianza frente: 2587.50299877295
varianza de clase: 2587.50299877295
varianza fondo: 0
varianza frente: 2587.50299877295
varianza de clase: 2587.50299877295
varianza fondo: 0
varianza frente: 2587.50299877295
varianza de clase: 2587.50299877295
varianza fondo: 0
varianza frente: 2587.50299877295
varianza de clase: 2587.50299877295
varianza fondo: 0
varianza frente: 2587.50299877295
varianza de clase: 2587.50299877295
varianza fondo: 0
varianza frente: 2587.50299877295
varianza de clase: 2587.50299877295
varianza fondo: 0
varianza frente: 2587.50299877295
varianza de clase: 2587.50299877295
varianza fondo: 0
varianza frente: 2587.50299877295
varianza de clase: 2587.319956613533
varianza fondo: 0.0
varianza frente: 2587.3495664794823
varianza de clase: 2587.0196896544107
varianza fondo: 0.234375
varianza frente: 2587.0986344865482
varianza de clas

In [7]:
#imprimimos las imagenes para ver como ha resultado todo hasta el momento

cv2.imshow("original", imagen)
cv2.imshow("umbralada", imagenUmbralada)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [52]:
#aplicamos morfologia para cerrar grietas obtenidas en el umbral de OTSU
estructura = np.ones((35,35), np.uint8)
imagenMorph = cv2.morphologyEx(imagenUmbralada, cv2.MORPH_OPEN, estructura)

#visualizamos

cv2.imshow("umbralada", imagenUmbralada)
cv2.imshow("clausuradda", imagenMorph)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
#encontramos los contornos de la imagen ya procesada

contornos, jerarquia = cv2.findContours(imagenMorph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#al menos en este experimento, a base de ejecuciones, determinamos manualmente que los centroides de los tomates que buscamos, son el 2 y el 0
#así que ejecutamos el algoritmo para obtener esos centroides y poder trabajar la parte restante

#creemos tambien una lista que me guarde los centroides de interes

centroides_interes = []
interes = [0,2]

for i in interes:
    centroide = contornos[i]
    momentos = cv2.moments(centroide)

    coordX = int(momentos["m10"]/momentos["m00"])
    coordY = int(momentos["m01"]/momentos["m00"])

    #guardamos en el arreglo el centroide
    centroides_interes.append([coordX, coordY])

    #dibujamos la ubicacion de los centoides

    #cv2.circle(imagenMorph, (coordX, coordY), 5, (0,255,0), -1)  <- Esta linea dibuja el centroide sobre la imagen, para comprobar donde esta el centroide descomente esta linea

#cisualizamos los centorides

#cv2.imshow("centroides", imagenMorph)

#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [35]:
#comenzamos a trabajar con el centrooide del tomate 2 <- ubicado en el centroide  2

#como este tomate se encuentra alineado al eje horizontal, este es el caso más facil

contadorPixeles = 0
pixelCentral = centroides_interes[1] #<- recordemos que primero almacenamos el centroide 4 y luego el 2, por eso el indice 1
pixel = pixelCentral
bandera = True
direccion = True

print(pixel)
print(imagenMorph[160, 375])

while(bandera or direccion):
    
    if imagenMorph[pixel[1], pixel[0]] > 0:
        contadorPixeles+=1
        #imagenMorph[pixel[1], pixel[0]] = 0 # <- Esto pinta la imagen ya umbralada y con morfologia aplicada, así que conviene solo usarla para probar
        bandera = True

        if(direccion):
            pixel[0] -=1 #<- movemos horizontalmente el pixel analizado, ya que la distancia que queremos encontrar es está en el eje de las absisas
        else:
            pixel[0] +=1
    else:
        if(direccion == True):
            direccion = False
            pixel = [pixelCentral[0]+1, pixelCentral[1]]
        else:
            bandera = False
        

#imprimimos el contador y la imagen resultante para comprobar resultados

print("el tomate 2 mide de punta a punta {} pixeles".format(contadorPixeles))
cv2.imshow("centroides", imagenMorph)

cv2.waitKey(0)
cv2.destroyAllWindows()

#aplicamos el proceso por segunda vez, pero esta vez haciendola cuenta del centro hasta el borde derecho

'''pixel = pixelCentral
bandera = True

while(bandera):
    
    if imagenMorph[pixel[0], pixel[1]] > 0:
        contadorPixeles+=1
        bandera = True
        pixel[0] +=1 #<- movemos horizontalmente el pixel analizado, ya que la distancia que queremos encontrar es está en el eje de las absisas
    else:
        bandera = False'''        


[306, 155]
255
el tomate 2 mide de punta a punta 130 pixeles


'pixel = pixelCentral\nbandera = True\n\nwhile(bandera):\n    \n    if imagenMorph[pixel[0], pixel[1]] > 0:\n        contadorPixeles+=1\n        bandera = True\n        pixel[0] +=1 #<- movemos horizontalmente el pixel analizado, ya que la distancia que queremos encontrar es está en el eje de las absisas\n    else:\n        bandera = False'

In [54]:
#Ahora aplicamos el procesamiento al segundo tomate, ubicado en el centroide 0

contadorPixeles = 0
pixelCentral = centroides_interes[0]
pixel = pixelCentral
tomate = contornos[0] #<- corresponde al numero de controno [0,4] que fueron los que encontramos al preprocesar la imagen
#tenemos que encontrar uno de los pixeles más alejados del pixel central, y para eso, sacando uno de los dos, ya tenemos la panediente, con la cual podemos sacar el otro, y dibujar la linea

#primero usamos un bucle para que el pixelse aleje a la izquierda del pixel central tanto como sea posible

mayor_distancia = 0
pixelHonorario = []

for pixel in tomate:
    print(pixel[0])
    distancia = np.sqrt((pixelCentral[0]-pixel[0][0])**2 + (pixelCentral[1]-pixel[0][1])**2)

    if mayor_distancia == 0:
        mayor_distancia = distancia
        pixelHonorario = pixel[0]
    else:
        if(distancia>mayor_distancia):
            mayor_distancia = distancia
            pixelHonorario = pixel[0]
        else:
            mayor_distancia = mayor_distancia
            pixelHonorario = pixelHonorario

#para comprobar tracemos una linea entre el pixel con la mayor distancia del centro
cv2.line(imagenMorph, pixelHonorario, pixelCentral, (0,0,0), 5)

#ahora que tenemos la mitad de la distancia mas larga, el dobe de esa distancia, sera la distancia en pixeles de punta a punta del tomate
distanciaTotal = 2*mayor_distancia

#finalmente, para terminar el trazado de la linea, al pixel que localizamos, le sumamos el doble de su suma conel pixel central
segundoExtremo = [pixelCentral[0]+ (pixelCentral[0]-pixelHonorario[0]), pixelCentral[1] + (pixelCentral[1]-pixelHonorario[1])]

#imprimamos su linea, a ver que ocurre
cv2.line(imagenMorph, pixelCentral, segundoExtremo, (0,0,0), 5)

#listo, aparentemente ya esta todo, habra que haerle algunos ajustes con la imagenn completa

print("La distancia del tomate 4 de extremo a extremo en pixeles es de {} pixeles".format(distanciaTotal))
#imprimimos la imagen resultante

cv2.imshow("linea", imagenMorph)

cv2.waitKey(0)
cv2.destroyAllWindows()

#print(tomate[0])


[257 346]
[256 347]
[254 347]
[253 348]
[252 348]
[251 349]
[250 349]
[249 350]
[247 350]
[245 352]
[244 352]
[243 353]
[242 353]
[241 354]
[240 354]
[238 356]
[237 356]
[235 358]
[234 358]
[229 363]
[228 363]
[220 371]
[210 371]
[210 372]
[209 373]
[207 373]
[207 388]
[204 391]
[204 392]
[203 393]
[203 394]
[202 395]
[202 397]
[200 399]
[200 400]
[199 401]
[199 403]
[198 404]
[198 406]
[197 407]
[197 408]
[195 410]
[195 446]
[196 447]
[196 449]
[197 450]
[197 456]
[198 457]
[199 457]
[200 458]
[200 459]
[201 460]
[201 461]
[205 465]
[206 465]
[207 466]
[207 479]
[208 480]
[209 480]
[212 483]
[212 484]
[248 484]
[249 483]
[260 483]
[261 482]
[263 482]
[264 481]
[267 481]
[268 480]
[289 480]
[289 466]
[290 465]
[290 464]
[292 462]
[299 462]
[300 461]
[301 461]
[301 460]
[302 459]
[303 459]
[305 457]
[305 456]
[306 455]
[325 455]
[325 434]
[326 433]
[326 391]
[325 390]
[325 389]
[324 388]
[324 387]
[318 381]
[318 380]
[316 378]
[316 377]
[315 376]
[315 375]
[312 372]
[312 371]
[310 369]
